<div style="direction:rtl;line-height:300%;"><font face="XB Zar" size=5>
<div align=center>
<font face="IranNastaliq" size=30>
<p></p><p></p>
<p></p>
</font>
<p></p>
<font color=blue>
Modern Information Retreival<br>
</font>
<p></p>
<font color=green>
HW2 -   Classification and clustering on Advertising information</font>
<p></p>
<font color=#FF7500>
<br>
Sharif U of Tech<br>
</font>
</div>
</font>
</div>


<div style="direction:ltr;line-height:300%;"><font face="XB Zar" size=5>
<font color=red size=7>
<p></p>
<div align=center>intro </div>
</font>
<hr>
In the second phase of the project, you will need to implement a classification and clustering system for the data set of ads. The project consists of 2 sections and its structure is as follows:    <br>
    <ol>
<li>
Classified Ads<ul>
    <li>Run tf-idf</li>
    <li>Run word2vec </li>
    <li>k-NN Implementation and similarity criteria</li>
    <li>Implement Naive Bayes </li>
    <li>Implementation of evaluation criteria</li>
    <li>Illustration and analysis of methods accuracy</li>
    <li>Predict the new ad classification </li>
</ul>
</li>
        <li>
Clustering ads<ul>
    <li>convert the ad to vector</li>
    <li>Implementation of the K-Means Clustering Algorithm</li>
    <li>Illustration and analysis of the algorithm</li>
</ul>
</li>

</font></div>




<div style="direction:ltr;line-height:300%;"><font face="XB Zar" size=5>
<font color=red size=7>
<p></p>
<div align=center>Data set</div>
</font>
<hr>
In this project, digital ad inventory will be used from the site data set and the "Divar" app. This collection of documents consists of three parts.<br>
The first part is in the file
`train.json`
Includes 80,000 ad text. Each ad contains 
ad id
(Id),
Title
(title)
Classified Ads
(cat)
And explain the ad
(desc)

The ad's classification and its identifier is a natural number
<br>
The second part is in the file
`judge.json`
     It is similar to the `train.json` file, except that the classification of this product is not specified.
     <br>
The third part is in the file
`word_embeddings.txt`
Contains
     Words of words are discussed further below.<br>
An example of a document:</font></div>




```
{
  "cat": 176,
  "title": "فتوکپی چهارکاره hp .بسیارنو",
  "desc": "فتوکپی چهارکاره hp.\n\nدستگاه چهارکاره شامل\nپرینتر.چاپگر.اسکن.فاکس\nبسیار نو \nکارکرد یک ماه \nدارای edf\nتخفیف هم می دم\nلازم به ذکر است قیمت نوی این دستگاه$NUMتومان است",
  "id": 8
}
```

<div style="direction:ltr;line-height:300%;"><font face="XB Zar" size=5>
<font color=red size=7>
<p></p>
<div align=center>Required libraries</div>
</font>
<hr>
The following libraries are used and you must install them:<ul>
    <li>plotly </li>
    <li>sklearn , t-SNE</li>
    <li> numpy </li>
    <li>hazm</li>
</ul>
</font></div>








In [ ]:
!pip install plotly sklearn numpy hazm

<div style="direction:rtl;line-height:300%;"><font face="XB Zar" size=5>
<font color=red size=7>
<p></p>
<div align=center>Classification</div>
</font>
<hr>
</font></div>

<div style="direction:ltr;line-height:300%;"><font face="XB Zar" size=5>

<ul>
           evaluation types

<li>
 accuracy
</li>
<li>
 precision  ,  recall , F1 
</li>
<li>
 F1 macro-averaging
    </li>
</ul>
</font>
</div>

<div style="direction:ltr;line-height:300%;"><font face="XB Zar" size=5>
<h3>
k-NN</h3>
To use the k-NN algorithm, a similarity criterion should be defined between documents. We will continue to process this document by converting documents to vector and using the standard distance metrics.</font>
</div>

<div style="direction:ltr;line-height:300%;"><font face="XB Zar" size=5>
    <h4>tf-idf</h4>
At the beginning of this section, we need to bring the documents to the ntn mode into the tf-idf vector space.
For fragmentation of sentences, we use the `word_tokenize` function of the hazm library</font>
</div>

In [2]:
import json

def load_data(data_path):
    docs = []
    with open(data_path,"r") as p:
        for l in p:
            docs.append(json.loads(l))
    return docs

docs = load_data("train.json")

In [3]:
from hazm import word_tokenize
import math

def get_words_idfs(docs):
    words = set()
    word_freq = {}
    for doc in docs:
        doc_words = set(word_tokenize(doc.get('title')))
        doc_words |= set(word_tokenize(doc.get('desc')))
        words |= doc_words
        for word in doc_words:
            if word in word_freq:
                word_freq[word] += 1
            else:
                word_freq[word] = 1
    idfs = {word: math.log((float)(len(docs)) / freq) for word, freq in word_freq.items()}
    return list(words), idfs

words, idfs = get_words_idfs(docs)

In [4]:
word_index = {}
for ind, word in enumerate(words):
    word_index[word] = ind

In [5]:
from scipy.sparse import csr_matrix
import numpy as np
from collections import Counter

def get_spars_matrix(docs):
    rows = []
    columns = []
    data = []
    for doc_ind, doc in enumerate(docs):
        tfs = Counter(word_tokenize(doc.get('title') + ' ' + doc.get('desc')))
        for word, tf in tfs.items():
            if word in word_index:
                rows.append(doc_ind)
                columns.append(word_index[word])
                data.append(tf)
    sparse_matrix = csr_matrix((np.array(data), (np.array(rows), np.array(columns))), shape=(len(docs), len(words)))
    return sparse_matrix

In [6]:
sparse_matrix = get_spars_matrix(docs)

In [7]:
from operator import add

def tf_idf_vector(document, words, idfs):
    vec = [0]*len(words)
    for doc in [document.get('title'), document.get('desc')]:
        tfs = Counter(word_tokenize(doc))
        document_vector = [0]*len(words)
        for word, tf in tfs.items():
            if word in words:
                ind = words.index(word)
                document_vector[ind] = tf * idfs[word]
        vec = list(map(add, vec, document_vector))

    return vec

In [8]:
a = tf_idf_vector({'title':'سلام', 'desc':'سلام'}, words, idfs)
print(len(words), len(a))

189878 189878


<div style="direction:ltr;line-height:300%;"><font face="XB Zar" size=5>
    <h4>word2vec</h4>
    
  <h4>Architecture </h4>
The neural network architecture used in this exercise is different from the original structure of the word2vec neural network.
This architecture is designed to accelerate the training of the model and use the negative sampling technique.
     The input of this two-word neural network are
     `input_target`
     And
     `input_context`
      . Then these two words are converted to their corresponding vector, their vectors are internally multiplied, and finally the sigmoid is calculated internally.
The variable `input_label` also specifies whether the two` target` and `context` characters appear together and have zero or one values.
<img src="arch.png"/>
To train this model, we need to get close and distant words as training data. For close words (`input_label = 1`), we use the normal structure of the skip-gram model.
So, define a window and consider each word for the words in its window as a positive input. For example, in the sentence<br/>
دانشگاه صنعتی شریف در خیابان آزادی است
<br/>
If we consider the window size 5, the following pairs will be created for the word "شریف":    <ul>
        <li>(شریف , دانشگاه)</li>
        <li>(شریف , صنعتی)</li>
        <li>(شریف , در)</li>
        <li>(شریف , خیابان)</li>
    </ul>
<br/>
For words far from each other, the negative sampling technique is implemented. To do this, you need to generate a number of negative examples (`input_label = 0`) for each window forward. To do this, just select a few words in the dictionary randomly and fit the number of duplicates in the dictionary.
In fact, in this random selection of words, the likelihood of occurrence of repetitive words should be greater. You can use the function to do this
<a href="https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.choice.html">`numpy.random.choice`</a>

    
    
   <h4>  `get_keras_model`:</h4>
    <br/>
The input of this function is the size of the dictionary and the size of the vectors, and its output is a `keras` model with embedding words used in the model.
<br/>
        
   <h4>  `train_word2vec`:</h4>
In this function, input data for the neural network is created and the model will be create.
Given that several hundred million data can be created for training, it is not necessary to store all of this data in an array, and we can use it to give the model a bit after reaching the reasonable size of the training data (`batch_size`) And then erase it. Due to a large amount of data, the training process is performed only for an iteration.
<br/>
                <h4>  `word2vec_vector`:</h4>
In this function we convert new documents into vector. We consider the vector of a document equal to the average vector of the words of that document.</font>
</div>

In [9]:
def clean_data(documents, min_count):
    doc_word_freq = []
    words_freq1 = {}
    for doc in documents:
        freq = {}
        words = word_tokenize(doc.get('title') + ' ' +doc.get('desc'))
        for word in words:
            if word in words_freq1:
                words_freq1[word] += 1
            else:
                words_freq1[word] = 1
            if word in freq:
                freq[word] +=1
            else:
                freq[word] = 1
        doc_word_freq.append(freq)

    unique_words = []
    docs_words = []
    words_freq2 = []
    words_count = 0
    for doc in doc_word_freq:
        doc_words = []
        for word, count in doc.items():
            if words_freq1[word] >= min_count:
                if word not in unique_words:
                    unique_words.append(word)
                    words_freq2.append(count)
                else:
                    words_freq2[unique_words.index(word)] += count
                doc_words.append(unique_words.index(word))
                words_count += count
        docs_words.append(doc_words)

    word_index = {word: i for i, word in enumerate(unique_words)}
    index_word = {i: word for i, word in enumerate(unique_words)}
    words_prob = [(float)(c)/words_count for c in words_freq2]

    return docs_words, word_index, index_word, words_prob

In [10]:
def generate_training_data(doc_words, window_size, negative_samples, words_prob):
    train_target = []
    train_context = []
    train_label = []
    left_size = math.floor((float)(window_size) /2)
    right_size = math.ceil((float)(window_size) /2)
    for i in range(len(doc_words)):
        right = max(0, i-left_size)
        left = min(len(doc_words), i+right_size)
        context = [doc_words[j] for j in range(right, left) if j != i]
        for con in context:
            train_target.append(doc_words[i])
            train_context.append(con)
            train_label.append(1)
        negative = []
        while len(negative) < negative_samples:
            candids = np.random.choice(len(words_prob), negative_samples-len(negative), p=words_prob)
            for candid in candids:
                if candid not in context and candid != doc_words[i]:
                    negative.append(candid)
        for n in negative:
            train_target.append(doc_words[i])
            train_context.append(n)
            train_label.append(0)

    return train_target, train_context, train_label

In [11]:
from keras.models import Model
from keras.layers import Input, Dense, Reshape, Dot
from keras.layers.embeddings import Embedding


def get_keras_model(vocab_size, dimension):
    """
    Build a keras model for word2vec

    :param vocab_size: count of unique tokens in vocabulary
    :param dimension: size of hidden dimensions in the neural net
    :return: a pair of keras model and the embedding used in the model
    """
    input_target = Input((1,))
    input_context = Input((1,))

    embedding = Embedding(vocab_size, dimension, input_length=1, name='embedding')
    target = embedding(input_target)
    target = Reshape((dimension, 1))(target)
    context = embedding(input_context)
    context = Reshape((dimension, 1))(context)
    dot_product = Dot(axes=1,normalize=True)([target, context])
    dot_product = Reshape((1,))(dot_product)
    output = Dense(1, activation='sigmoid')(dot_product)
    model = Model(inputs=[input_target, input_context], outputs=output)
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model, embedding

/home/hadi/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [12]:
def train_word2vec(documents,
                   dimension=50,
                   negative_samples=5,
                   window_size=5,
                   min_count=10,
                   batch_size=4096):
    """
    Train word embeddings from documents
    
    :param documents: a list of json objects containing training documents 
    :param dimension: size of hidden vectors in word2vec
    :param negative_samples: number of negative samples
    :param window_size: skip-gram window size
    :param min_count: ignore words with frequency less than or equal to min_count 
    :param batch_size: batch size for training
    :return: the embedding matrix with word_index to convert words to their indexes
    """
    # Clean vocabulary and drop rare words
    # Map words to integers in [0, vocab_size)
    docs_words, word_index, index_word, words_prob = clean_data(documents, min_count)
    print('Word NUM:', len(word_index))
    
    
    model, embedding = get_keras_model(len(words_prob), dimension)
    
    train_target = [] # indices of target words
    train_context = [] # indices of context words
    train_label = [] # labels of training data (0 or 1)
    training_data_num = 0
    for doc in documents:
        # Implement sliding window for positive inputs
        # Implement negative sampling for negative inputs
        # Fill train_ variables
        train_target, train_context, train_label = generate_training_data(docs_words[doc.get('id')-1], window_size, negative_samples, words_prob)
        training_data_num += len(train_label)
        
        while len(train_label) > batch_size:
            model.train_on_batch(x=[train_target[:batch_size],train_context[:batch_size]],y=train_label[:batch_size])
            
            #Remove the batch
            train_target=train_target[batch_size:]
            train_context=train_context[batch_size:]
            train_label=train_label[batch_size:]
    print('Training data NUM:', training_data_num)
    word_vectors = embedding.get_weights()[0] # a 2-D numpy array with word embeddings
    return word_vectors, word_index

In [13]:
word_vectors, word_index = train_word2vec(docs[:2000])

Word NUM: 1724
Training data NUM: 772089


In [14]:
word_vectors.shape

(1724, 50)

<div style="direction:ltr;line-height:300%;"><font face="XB Zar" size=5>
<ul>
    <li>
        For each of the words below, we will find 10 words that are most similar with the cosine similarity:
        <ul>
            <li>دوربین</li>
            <li>iphone</li>
            <li>آیفن</li>
            <li>سامسونگ</li>
            <li>مگاپیکسل</li>
            <li>تشکر</li>
            <li>بیمه</li>
            <li>پرینتر</li>
            <li>xiaomi</li>
        </ul>
    </li>
</ul>

</font>
</div>

In [49]:
from sklearn.metrics.pairwise import cosine_distances, euclidean_distances

can_words = ['دوربین', 'iphone', 'آیفون', 'سامسونگ', 'مگاپیکسل', 'تشکر', 'بیمه', 'پرینتر', 'xiaomi']
for word in can_words:
    if word in word_index:
        ind = list(word_index.keys()).index(word)
        vec1 = word_vectors[ind]
        min_dis = math.inf
        min_word = ''
        for ind2, vec2 in enumerate(word_vectors):
            if ind2 != ind:
                dis = cosine_distances([vec1], [vec2])[0][0]
                if dis < min_dis:
                    min_dis = dis
                    min_word = list(word_index.keys())[ind2]
        print(word, ':', min_word)
    

دوربین : شکسته
iphone : البته
آیفون : منوی
سامسونگ : لبخند
مگاپیکسل : آن
تشکر : GHz
پرینتر : بسته


<div style="direction:ltr;line-height:300%;"><font face="XB Zar" size=5>
`word2vec_vector` function: The input of a function is a dictionary containing the `title` and` desc` fields.
If we do not have the input word in the `word_vectors_ 'of the dictation, we will not consider it.
We take the vector of each document into the mean of the vector of words within it.
The output is an array of length `dimension` that represents the document vector.
</font>
</div>

In [56]:
dimension=50
    
def word2vec_vector(document):
    """
    Convert a document to vector by tokenizing and embedding it using the trained word embeddings
    :param document: a single document
    :return: the corresponding vector for document
    """
    doc_vec = np.array([0]*dimension)
    num = 0
    for doc in [document.get('title'), document.get('desc')]:
        tokens = word_tokenize(doc)
        for word in tokens:
            if word in word_index:
                word_vec = [0.0]*(len(word_index))
                word_vec[word_index[word]] = 1.0
                word_vec = np.array(word_vec)
                word_vec = np.dot(word_vec, word_vectors)
                doc_vec = doc_vec + word_vec
                num += 1
    if num > 0:
        doc_vec = doc_vec / num
    return doc_vec

In [57]:
word2vec_vector(docs[100])

array([-6.42852712e-03,  3.01126036e-03, -8.64254492e-03, -1.87315547e-03,
        1.36403562e-03,  2.31713556e-03, -4.34215135e-03, -6.62350073e-04,
        7.30844028e-03, -8.07425325e-04,  2.21851168e-04,  5.76236099e-03,
       -2.95648592e-03,  1.02901217e-02,  3.50088699e-05, -5.47677651e-03,
       -1.47643558e-03,  4.84769287e-03, -6.20206068e-03, -1.33921074e-03,
        2.39976650e-03,  6.86120764e-03, -9.75651303e-04, -1.45131103e-02,
       -5.42440774e-03, -4.12332062e-03, -1.33480200e-03, -6.38015909e-03,
        4.42114563e-03,  5.27252693e-04, -9.09326180e-03,  5.69084686e-03,
       -2.33047542e-03, -1.10440933e-03,  6.89376577e-03, -9.29801779e-03,
       -8.64878261e-03, -4.61706496e-03,  1.20854558e-02,  1.25855026e-04,
       -6.60547055e-03, -6.06380712e-03,  1.34526068e-02,  1.92793677e-04,
        1.49496004e-02,  6.54297349e-03,  1.06996002e-02, -1.20449089e-03,
        2.62161294e-03,  3.25948954e-03])

<div style="direction:ltr;line-height:300%;"><font face="XB Zar" size=5>
We implement the k-NN algorithm and execute for each of the following parameters:
<ul>
<li>
$k$
<ul>
<li>1</li><li>3</li><li>5</li>
</ul>
</li>
<li>
document's vector 
    <ul>
<li>tf-idf</li><li>word2vec</li>
</ul>
</li>

<li>
Similarity criterion<ul>
<li>Cosine</li><li>Euclidean</li><li>Internal multiplication</li>
</ul>
</li>


</ul>
<br>
Each time, we compute all the required evaluation criteria and ultimately analyze and evaluate this algorithm for various parameters.</font>
</div>


In [58]:
from sklearn.metrics.pairwise import cosine_distances, euclidean_distances

def get_distance(v1, v2, similarity_measure):
    if similarity_measure == 'cosine':
        return cosine_distances(v1, v2)[0]
    elif similarity_measure == 'euclidean':
        return euclidean_distances(v1, v2)[0]
    return v2.dot(v1[0])

In [59]:
word2vec_matrix = []
for doc in docs:
    word2vec_matrix.append(word2vec_vector(doc))

KeyboardInterrupt: 

In [60]:
def get_vectors(method):
    if method == 'word2vec':
        return word2vec_matrix
    return sparse_matrix

In [61]:
def get_vector(doc, method):
    if method == 'word2vec':
        return word2vec_vector(doc)
    return tf_idf_vector(doc, words, idfs)

In [62]:
k = 5 # or 1 or 3
similarity_measure = 'cosine' # or 'euclidean' or 'dot'
vectors = 'word2vec' # 'tf_idf' or 'word2vec'

def knn(document, k=5, similarity_measure='cosine', vectors='word2vec'):
    # document: سندی که قرار است دسته‌بندی شود در فضای برداری
    # predicted_class: کلاسی که سند مورد نظر در آن دسته قرار میگیرد.
    # TODO
    vecs = get_vectors(vectors)
    vec1 = get_vector(document, vectors)
    dis = get_distance([vec1], vecs, similarity_measure)
    h = dis.argsort()[-k:]
    cats = []
    for ind in h:
        cats.append(docs[ind].get('cat'))
    predicted_class = max(set(cats), key=cats.count)
            
    return predicted_class

In [49]:
knn(docs[1])

176

<div style="direction:ltr;line-height:300%;"><font face="XB Zar" size=5>
    <h3> Naive Bayes</h3>
</font>
</div>

In [63]:
def trainNB(train_documents):
    # train_documents: All the test data in vector space
    
    # class_priority: The probability of each class to be selected as the document class
    # conditional_probability: The probability of each word per class
    
    word_cat_freq = {}
    cat_word_freq = {}
    cat_freq = {}
    all_words = set()
    all_cats = set()
    
    for doc in train_documents:
        tokens = word_tokenize(doc.get('title'))
        tokens.extend(word_tokenize(doc.get('desc')))
        cat = doc.get('cat')
        
        all_cats.add(cat)
        if cat in cat_freq:
            cat_freq[cat] += 1
        else:
            cat_freq[cat] = 1
        
        for word in tokens:
            all_words.add(word)
            if word in cat_word_freq:
                cat_word_freq[cat] += 1
            else:
                cat_word_freq[cat] = 1
            if word in word_cat_freq:
                if cat in word_cat_freq[word]:
                    word_cat_freq[word][cat] += 1
                else:
                    word_cat_freq[word][cat] = 1
            else:
                word_cat_freq[word] = {}
                word_cat_freq[word][cat] = 1
        
        words_num = len(all_words)
        
        class_priority = {}
        conditional_probability = {}
        
        for cat, count in cat_freq.items():
            class_priority[cat] = (float)(count) / len(train_documents)
        
        for word in word_cat_freq:
            conditional_probability[word] = {}
            for cat in all_cats:
                conditional_probability[word][cat] = (float)(word_cat_freq[word].get(cat, 0)+1) / (cat_word_freq[cat] + words_num)
    
    return class_priority, conditional_probability

In [64]:
import operator

class_priority, conditional_probability = trainNB(docs[:8000]) 

def predictNB(document):
    # document: Document to be classified into vector space
    
    # predicted_class: The class in which the document is intended.
    
    # TODO
    tokens = word_tokenize(document.get('title'))
    tokens.extend(word_tokenize(document.get('desc')))
    probs = {}
    
    for cat, cat_prob in class_priority.items():
        prob = math.log(cat_prob)
        for word in tokens:
            if word in conditional_probability :
                prob += math.log(conditional_probability[word][cat])
        probs[cat] = prob
    
    predicted_class = max(probs.items(), key=operator.itemgetter(1))[0]
    return predicted_class

KeyboardInterrupt: 

In [96]:
predictNB(docs[1])

119

<div style="direction:rtl;line-height:300%;"><font face="XB Zar" size=5>
We use this code to test the classifier</font>
</div>

In [28]:
from IPython.display import display,clear_output
import ipywidgets as widgets


title_text_widget = widgets.Text(
    value='',
    placeholder='عنوان آگهی',
    description='Title:',
    disabled=False
)

desc_text_widget = widgets.Text(
    value='',
    placeholder='متن آگهی',
    description='Desc:',
    disabled=False
)


out = widgets.Output()

def text_submit(event):
    with out:
        clear_output(wait=False)
        print('k-NN result', knn({'title':title_text_widget.value,'desc':desc_text_widget.value}))
        print('Naive Bayes result', predictNB({'title':title_text_widget.value,'desc':desc_text_widget.value}))
        

title_text_widget.on_submit(text_submit)
desc_text_widget.on_submit(text_submit)
display(title_text_widget)
display(desc_text_widget)
display(out)

Text(value='', description='Title:', placeholder='عنوان آگهی')

Text(value='', description='Desc:', placeholder='متن آگهی')

Output()

<div style="direction:ltr;line-height:300%;"><font face="XB Zar" size=5>
We also get Confusion matrix in each section and compute all the criteria for the evaluation.
    <br>
`confusion_matrix`: 2D array
<br>
`confusion_matrix[i][j]`: The number of documents in the class 𝑖 and recognized in class 𝑗.
<br>
    `target_class`: The desired class
</font>
</div>


In [29]:
all_cats = list(cat_names.keys())
classes_title = list(cat_names.values())

In [65]:
# confusion_matrix_example: [[2,4],[3,5]]

def create_confusion_matrix(test_documents, classifier, k=5, similarity_measure='cosine', vectors='word2vec'):
    # test_documents تمامی دادگان تست
    # classifier: تابع دسته بندی 
    # classifier_example: knn_tf_idf(document) , predictNB(document)
    
    # TODO
    confusion_dict = {}
    for cat1 in all_cats:
        confusion_dict[cat1] = {}
        for cat2 in all_cats:
            confusion_dict[cat1][cat2] = 0
            
    for doc in test_documents:
        src_cat = doc.get('cat')
        dst_cat = -1
        if 'predictNB' in classifier:
            dst_cat = predictNB(doc)
        else:
            dst_cat = knn(doc, k, similarity_measure, vectors)
        confusion_dict[src_cat][dst_cat] += 1
        
    confusion_matrix = []
    for i in range(len(all_cats)):
        confusion_matrix.append([])
        for j in range(len(all_cats)):
            confusion_matrix[i].append(confusion_dict[all_cats[i]][all_cats[j]])
        
    return confusion_matrix

def calculate_precision(confusion_matrix, target_class):
    # TODO
    dst_ind = all_cats.index(target_class)
    total = 0
    for i in range(len(confusion_matrix)):
        total += confusion_matrix[i][dst_ind]
    if total == 0:
        return 0.0
    precision = (float)(confusion_matrix[dst_ind][dst_ind]) / total
    return precision

def calculate_recall(confusion_matrix, target_class):
    # TODO
    src_ind = all_cats.index(target_class)
    total = 0
    for i in range(len(confusion_matrix)):
        total += confusion_matrix[src_ind][i]
    if total == 0:
        return 0.0
    racall = (float)(confusion_matrix[src_ind][src_ind]) / total
    return racall

def calculate_F1_for_class(confusion_matrix, target_class):
    # TODO
    precision = calculate_precision(confusion_matrix, target_class)
    recall = calculate_recall(confusion_matrix, target_class)
    F1 = 0.0
    if (precision + recall) > 0.0:
        F1 = (2.0 * precision * recall) / (precision + recall)
    return F1

def calculate_F1(confuision_matrix):
    # TODO
    total = 0.0
    for cat in all_cats:
        total += calculate_F1_for_class(confusion_matrix, cat)
    F1 = total / len(all_cats)
    return F1

def calculate_accuracy(confusion_matrix):
    # TODO
    total = 0
    correct = 0
    for i in range(len(confusion_matrix)):
        for j in range(len(confusion_matrix)):
            total += confusion_matrix[i][j]
            if i == j:
                correct += confusion_matrix[i][j]
    accuracy = (float)(correct) / total
    return accuracy


In [71]:
confusion_matrix = create_confusion_matrix(docs[:80000], 'predictNB')

In [72]:
calculate_F1(confusion_matrix)

0.3156039221481347

In [101]:
for k in [1,3,5]:
    for vectors in ['tf_idf', 'word2vec']:
        for similarity_measure in ['cosine', 'euclidean', 'dot']:
            confusion_matrix = create_confusion_matrix(docs[:100], 'knn', k, vectors, similarity_measure)
            print('K:', k, 'Vectors:', vectors, 'Similarity measure:', similarity_measure)
            print('accuracy', calculate_accuracy(confusion_matrix))
            print('F1-macro-avg:', calculate_F1(confusion_matrix))
            for cat in all_cats:
                print('*** Category:', cat)
                print('precsion:', calculate_precision(confusion_matrix, cat))
                print('recall:', calculate_recall(confusion_matrix, cat))
                print('F1:', calculate_F1_for_class(confusion_matrix, cat))
            print('############################')

K: 1 Vectors: tf_idf Similarity measure: cosine
accuracy 0.32
F1-macro-avg: 0.276755600461944
*** Category: 25
precsion: 0.2
recall: 0.25
F1: 0.22222222222222224
*** Category: 26
precsion: 0.0
recall: 0.0
F1: 0.0
*** Category: 32
precsion: 0.45454545454545453
recall: 0.4166666666666667
F1: 0.43478260869565216
*** Category: 34
precsion: 0.3
recall: 0.6
F1: 0.4
*** Category: 36
precsion: 1.0
recall: 0.4375
F1: 0.6086956521739131
*** Category: 117
precsion: 1.0
recall: 0.16666666666666666
F1: 0.2857142857142857
*** Category: 119
precsion: 0.17777777777777778
recall: 0.5
F1: 0.26229508196721313
*** Category: 176
precsion: 0.0
recall: 0.0
F1: 0.0
*** Category: 177
precsion: 0.5
recall: 0.3333333333333333
F1: 0.4
*** Category: 178
precsion: 0.25
recall: 0.1111111111111111
F1: 0.15384615384615383
############################
K: 1 Vectors: tf_idf Similarity measure: euclidean
accuracy 0.32
F1-macro-avg: 0.276755600461944
*** Category: 25
precsion: 0.2
recall: 0.25
F1: 0.22222222222222224
*** C

In [102]:
confusion_matrix = create_confusion_matrix(docs[:80000], 'predictNB')
print('accuracy', calculate_accuracy(confusion_matrix))
print('F1-macro-avg:', calculate_F1(confusion_matrix))
for cat in all_cats:
    print('*** Category:', cat)
    print('precsion:', calculate_precision(confusion_matrix, cat))
    print('recall:', calculate_recall(confusion_matrix, cat))
    print('F1:', calculate_F1_for_class(confusion_matrix, cat))
print('############################')

accuracy 0.8739875
F1-macro-avg: 0.8724886838541549
*** Category: 25
precsion: 0.8686430166211858
recall: 0.8726479750778816
F1: 0.8706408901597563
*** Category: 26
precsion: 0.8080333854981743
recall: 0.9785217940619078
F1: 0.8851428571428571
*** Category: 32
precsion: 0.9598028062191885
recall: 0.9372916923836564
F1: 0.9484136897327005
*** Category: 34
precsion: 0.9618482407799915
recall: 0.8609916519099419
F1: 0.9086297804178068
*** Category: 36
precsion: 0.9222040095826504
recall: 0.9126528574993761
F1: 0.9174035747883349
*** Category: 117
precsion: 0.8937129605771709
recall: 0.8631330098295383
F1: 0.8781568453699602
*** Category: 119
precsion: 0.6324263819528155
recall: 0.9178955261184704
F1: 0.7488784665579118
*** Category: 176
precsion: 0.9852384557153672
recall: 0.9728416593995266
F1: 0.9790008148937503
*** Category: 177
precsion: 0.8597084301711388
recall: 0.5080534398801348
F1: 0.6386752472139381
*** Category: 178
precsion: 0.9862143532909853
recall: 0.9162481165243597
F1: 0.

<div style="direction:ltr;line-height:300%;"><font face="XB Zar" size=5>
    <h3>
        display Confusion Matrix  
    </h3>
At this point, we show the matrix of confluence, and then we find the classes that have the most error.    <br>
</font>
</div>

In [33]:
from plotly.offline import iplot, init_notebook_mode
from plotly import graph_objs as go

init_notebook_mode(connected=True)


iplot([go.Heatmap(z=confusion_matrix, x=classes_title, y=classes_title)])

<div style="direction:LTR;line-height:300%;"><font face="XB Zar" size=5>
<font color=red size=7>
<p></p>
<div align=center>Clustering</div>
</font>
<hr>
Clustering algorithms are used to analyze exploratory data and to find abnormal behaviors. One of the traditional methods for clustering textual data is using embedding algorithms and converting sentences into vectors of several hundreds. The embedding operation is performed for existing databases and the vector of each word is in the `word_embeddings.txt` file. Each line of this file comes with a word and then 100 digits, indicating the values ​​of the 100-word vector of the word. These values ​​are separated by the tab character. Given the vector of words, we construct the vector of documents and then execute the K-Means clustering algorithm on the documents.
<ol>

</ol>
</font></div>

<div style="direction:ltr;line-height:300%;"><font face="XB Zar" size=5>
- `embed_doc`: The input of a function is a dictionary containing the` title` and `desc` fields. Use the `word_tokenize` function of the hazm library to convert a string to a token. If we do not have the input word in the embedding list, we will not consider it. We take the vector of each document into the mean of the vector of words within it. The output is an array of 100, representing the vector of the document.
</font>
</div>

In [50]:
def load_embedings(path):
    embed_dict = {}
    with open(path, 'r') as my_file:
        for line in my_file:
            e = line.split('\t')
            embed_dict[e[0]] = [float(x) for x in e[1:]]
    return embed_dict

In [51]:
embed_dict = load_embedings('word_embeddings.txt')

In [52]:
from hazm import word_tokenize

def embed_doc(doc):
    text = doc['title'] + doc['desc']
    # TODO
    tokens = word_tokenize(text)
    embedding = [0.0]*100
    words_num = 0
    for word in tokens:
        if word in embed_dict:
            embedding= list(map(add, embedding, embed_dict[word]))
            words_num += 1
    if words_num > 0:
        embedding = [x / words_num for x in embedding]
    return embedding

embedding = embed_doc({'title':'سلام','desc':''})
assert len(embedding) == 100
print(embedding)

[0.03755388, -0.07322507, -0.25913474, 0.12415095, -0.11877345, -0.27827236, -0.18299168, 0.23008107, -0.039894868, -0.11424236, -0.31247956, -0.3502329, 0.30387092, 0.08331432, 0.14547259, -0.018599035, -0.08134301, 0.007101107, -0.09190335, 0.11664822, 0.46474212, -0.08209793, -0.1739652, 0.27787, -0.17775233, -0.15752481, 0.21262208, 0.118998416, 0.46111763, -0.15321721, 0.24574268, 0.16088983, 0.26111802, 0.32789925, 0.08191252, -0.32209042, 0.27611172, 0.12858805, 0.111668654, -0.27019724, 0.030425467, -0.063528985, 0.09130709, -0.15464024, -0.15617853, 0.14902186, 0.19106899, 0.279117, 0.3100722, -0.34242207, -0.031617142, -0.18922628, 0.14450774, -0.3961263, 0.079797685, 0.09358401, 0.08588229, -0.39507827, -0.34378484, -0.2230974, 0.085394055, -0.22743808, -0.47187024, -0.020511322, -0.02210253, 0.016363837, 0.07980231, 0.056026015, 0.21303584, -0.23450588, 0.6692689, 0.2311996, -0.053501617, -0.21110526, 0.12559295, -0.2188785, 0.050084706, 0.020516321, -0.17694162, -0.0336650

In [53]:
docs_vectors = [embed_doc(d) for d in docs[:1000]]

<div style="direction:ltr;line-height:300%;"><font face="XB Zar" size=5>
`kmeans`: input is the function of the array of vectors, the number of clusters, and the number of times the algorithm is executed. The output of the algorithm is the cluster number assigned to each vector and cluster centers.
</font>
</div>

In [54]:
import random

def kmeans(vectors, k, epochs):
    #TODO
    dup = []
    for v in vectors:
        for i in v:
            dup.append(i)
    centroids = []
    samples = random.sample(range(len(vectors)),k)
    centroids = [vectors[s] for s in samples]
    
    labels = []
    for i in range(epochs):
        labels = []
        all_labels = set()
        for v in vectors:
            min_dis = math.inf
            min_ind = -1
            for ind in range(len(centroids)):
                dis = get_distance([v], [centroids[ind]], 'euclidean')
                if dis < min_dis:
                    min_dis = dis
                    min_ind = ind
            labels.append(min_ind)
            all_labels.add(min_ind)
            
        all_labels = list(all_labels)
        labels = [all_labels.index(l)+1 for l in labels]
        
        new_centroids = [[0.0]*len(vectors[0]) for j in range(k)]
        new_num = [0]*k
        for j in range(len(vectors)):
            label = labels[j]-1
            new_centroids[label] = list(map(add, new_centroids[label], vectors[j]))
            new_num[label] += 1
        for j in range(len(new_centroids)):
            centroids[j] = [x/new_num[j] for x in new_centroids[j]]
        
    assert len(centroids) == k
    assert len(labels) == len(vectors)
    return labels,centroids


In [76]:
labels, centroids = kmeans(docs_vectors, 3, 3)

In [66]:
len(centroids)

3

<div style="direction:ltr;line-height:300%;"><font face="XB Zar" size=5>
With the help of the code, we bring the 100-dimensional vectors of two dimensional dimensions, and then we see our clustering results.
</font>
</div>

In [67]:
from sklearn.manifold import TSNE

tsne_output = TSNE(verbose=2).fit_transform(docs_vectors)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1000 samples in 0.002s...
[t-SNE] Computed neighbors for 1000 samples in 0.098s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1000
[t-SNE] Mean sigma: 0.121901
[t-SNE] Computed conditional probabilities in 0.240s
[t-SNE] Iteration 50: error = 60.8904610, gradient norm = 0.2535582 (50 iterations in 0.930s)
[t-SNE] Iteration 100: error = 56.8482819, gradient norm = 0.2011826 (50 iterations in 0.744s)
[t-SNE] Iteration 150: error = 56.2820511, gradient norm = 0.1793221 (50 iterations in 0.734s)
[t-SNE] Iteration 200: error = 56.1255379, gradient norm = 0.1725048 (50 iterations in 0.764s)
[t-SNE] Iteration 250: error = 56.0888786, gradient norm = 0.1732833 (50 iterations in 0.746s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 56.088879
[t-SNE] Iteration 300: error = 0.5603790, gradient norm = 0.0005974 (50 iterations in 0.742s)
[t-SNE] Iteration 350: error = 0.4863850, gradient norm = 0.00023

In [68]:
document_titles = [d.get('title') for d in docs[:1000]]

In [77]:
from plotly.offline import iplot, init_notebook_mode
from plotly import graph_objs as go

init_notebook_mode(connected=True)

iplot([go.Scattergl(x=tsne_output[:,0],
                    y=tsne_output[:,1],
                    mode='markers',
                    marker={'color':labels},
                    text=document_titles)])


<div style="direction:rtl;line-height:300%;"><font face="XB Zar" size=5>
<div align=center>
<font face="IranNastaliq" size=30>
<p></p>
    <br><br>
<font color=#FF7500> 
موفق باشید
💪
<br><br>

</font>
</div>
</font>
</div>